# MAL Basics
* This notebook is a high level introduction demonstrating that you can upload inferences from python objects, local files, and remote files.
* For complete examples see image_mal.ipynb or ner_mal.ipynb

* There are three ways to import annotations using the upload_annotations method.
    1. For images, the Model-assisted labeling workflow supports all annotation types, except Dropdown classification and classifications nested within classifications.
    2. For videos, the Model-assisted workflow only supports classifications at the frame-level.
    3. For text, the Model-assisted workflow supports Named entity recognition and text classification.

* Hints:
    * If you are importing more than 1,000 mask annotations at a time, consider submitting separate jobs, as they can take longer than other annotation types to import.
    * Wait until the import job is complete before opening the Editor to make sure all annotations are imported properly.

In [ ]:
!pip install labelbox
!pip install labelbox[data]
!pip install requests
!pip install ndjson
!pip install pathlib

In [14]:
from labelbox.schema.ontology import OntologyBuilder
import os
import uuid
from getpass import getpass

from labelbox import (
    Client, 
    LabelingFrontend, 
    OntologyBuilder, 
    Tool, 
    Classification, 
    Option
)

from labelbox.data.annotation_types import (
    LabelList,
    ImageData,
    MaskData,
    Rectangle,
    ObjectAnnotation,
    ClassificationAnnotation,
    Point,
    ClassificationAnswer,
    Radio,
    Mask,
    Label
)
from labelbox.data.serialization import NDJsonConverter


In [5]:
# If you don't want to give google access to drive you can skip this cell
# and manually set `API_KEY` below.

COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !pip install colab-env -qU
    from colab_env import envvar_handler
    envvar_handler.envload()

API_KEY = os.environ.get("LABELBOX_API_KEY")
if not os.environ.get("LABELBOX_API_KEY"):
    API_KEY = getpass("Please enter your labelbox api key")
    if COLAB:
        envvar_handler.add_env("LABELBOX_API_KEY", API_KEY)

In [6]:
# Only update this if you have an on-prem deployment
ENDPOINT = "https://api.labelbox.com/graphql"

In [23]:
client = Client(api_key=API_KEY, endpoint=ENDPOINT)

# Generate Annotations

In [76]:
image_urls = ["https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg"]

In [33]:
"""We can create a list of pre-defined annotations whether or not we have configured a project in Labelbox

The annotations in this example are pre-defined ObjectAnnotations.For an example on how to convert model outputs into Labelbox Annotations, 
refer to https://github.com/Labelbox/labelbox-python/blob/develop/examples/annotation_types/mal_using_annotation_types.ipynb
"""

label_list = LabelList()

for image_url in image_urls:
    image_data = ImageData(url = image_url)

    label_list.append(Label(
        data = image_data,
        annotations = [
            ObjectAnnotation(name='person', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=2275.82861328125, y=1352.3682861328125), end=Point(extra={}, x=2415.62060546875, y=1702.5)), classifications=[]), 
            ObjectAnnotation(name='person', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=2089.923828125, y=1251.9525146484375), end=Point(extra={}, x=2248.47265625, y=1679.967041015625)), classifications=[]), 
            ObjectAnnotation(name='person', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=1386.9190673828125, y=1041.0606689453125), end=Point(extra={}, x=1484.731201171875, y=1291.29541015625)), classifications=[])]            
    ))

# Project setup

In [ ]:
'''We can create a project and it's ontology using just our LabelList object!

Note: Creating nested ontologies are not supported 
through LabellList.get_ontology() at this time. To create
more complex ontologies, use the OntologyBuilder.
'''
project = client.create_project(name="image_mal_project")
dataset = client.create_dataset(name="image_mal_dataset")
editor = next(
    client.get_labeling_frontends(where=LabelingFrontend.name == 'editor'))
project.setup(editor, label_list.get_ontology().asdict())
project.datasets.connect(dataset)
project.enable_model_assisted_labeling()

# Prepare for upload
Uploading into Labelbox requires adhering to the format shown here: https://docs.labelbox.com/data-model/en/index-en#annotations.


To do this, we can upload annotations in two ways:

1. We can pass in a LabelList that contains our annotations
2. We can pass in an iterable of dicts

Prepare one of the ways below and run the upload to see the results in Labelbox


## Prepare via LabelList

For our LabelList, we need to update the following:

1. signed url for segmentation masks
2. data rows in labelbox
3. feature schema ids

In [ ]:
signer = lambda _bytes: client.upload_data(content=_bytes, sign=True)
label_list.add_url_to_masks(signer) \
         .add_url_to_data(signer) \
         .assign_feature_schema_ids(OntologyBuilder.from_project(project)) \
         .add_to_dataset(dataset, signer)

### Convert to the correct format (NDJson)
* We can use the NDJsonConverter to turn our label_list containng predictions into the correct format

In [61]:
annotations = list(NDJsonConverter.serialize(label_list))
print(annotations[0])

{'uuid': 'd25ffc73-e909-42e5-8566-3f9b7aa08134', 'dataRow': {'id': 'ckud7vaqv0qyl0yrffzq7gq6f'}, 'schemaId': 'ckud7v75e00b60y6dfli08zvq', 'classifications': [], 'bbox': {'top': 1352.3682861328125, 'left': 2275.82861328125, 'height': 350.1317138671875, 'width': 139.7919921875}}


## Prepare via iterable of dicts

In [75]:
'''We will generate a new annotation for this example.
For more details see image_mal.ipynb or ner_mal.ipynb
'''
annotations = [{
    "uuid": str(uuid.uuid4()),
    "schemaId": OntologyBuilder.from_project(project).tools[0].feature_schema_id,
    "dataRow": {
        "id": dataset.create_data_row(row_data=image_urls[0]).uid,
    },
    "bbox": {
        "top": int(30),
        "left": int(30),
        "height": 200,
        "width": 200
    }
}]

# The upload

### Perform the upload
* Choose one of the following to run
* Then run the cell with upload_job.wait_until_done()
* Once this completes, you can see the annotations if you go to the link that is printed out below

In [65]:
upload_task = project.upload_annotations(name=f"upload-job-{uuid.uuid4()}",
                                         annotations=annotations,
                                         validate=True)
# Wait for upload to finish
upload_task.wait_until_done()

In [74]:
# Review the upload status
for status in upload_task.statuses[:5]:
    print(status) 

print(
    "\nVisit the following and click start labeling to see the uploaded example:",
    f"https://app.labelbox.com/projects/{project.uid}")       

{'uuid': 'eb3ca696-8532-43ed-a57c-fbe3bdde8404', 'dataRow': {'id': 'ckud8sxhr00hc0y6d0ro994ff'}, 'status': 'SUCCESS'}

Visit the following and click start labeling to see the uploaded example: https://app.labelbox.com/projects/ckud7uzd44zq20y9a7smo51ur
